## Loukopoulos Orestis | f2822104

## Task 3

In [557]:
from pyspark.sql.session import SparkSession

spark = SparkSession.builder.appName("books").getOrCreate()

* We will load the json file in a dataframe(`df1`)

In [558]:
df1 = spark.read.json("Assignment Spark/books_5000.json")
type(df1)

pyspark.sql.dataframe.DataFrame

* In the following table we can see the columns that we will use for our regression model. The variable `average_rating` will be our response variable that we will try to predict, while the rest variables are the input variables (covariates).

In [559]:
df1.select(df1.average_rating, df1.language_code, df1.num_pages, df1.ratings_count, df1.publication_year).show()

+--------------+-------------+---------+-------------+----------------+
|average_rating|language_code|num_pages|ratings_count|publication_year|
+--------------+-------------+---------+-------------+----------------+
|          4.12|             |         |            1|                |
|          3.94|          fre|         |           16|            2016|
|          4.28|          eng|      146|           51|            2012|
|          4.05|          eng|         |            6|                |
|          4.06|        en-US|      272|           51|            1997|
|          3.44|             |      206|           46|            2007|
|          4.15|          eng|      224|           39|            2016|
|          3.16|             |      160|           38|            2016|
|          3.51|             |      160|           44|            2016|
|          4.00|             |      144|           32|            2016|
|          4.41|          kor|      212|          133|          

* We will check the data types of dataframe's variables.

In [560]:
df1.dtypes

[('asin', 'string'),
 ('authors', 'array<struct<author_id:string,role:string>>'),
 ('average_rating', 'string'),
 ('book_id', 'string'),
 ('country_code', 'string'),
 ('description', 'string'),
 ('edition_information', 'string'),
 ('format', 'string'),
 ('image_url', 'string'),
 ('is_ebook', 'string'),
 ('isbn', 'string'),
 ('isbn13', 'string'),
 ('kindle_asin', 'string'),
 ('language_code', 'string'),
 ('link', 'string'),
 ('num_pages', 'string'),
 ('popular_shelves', 'array<struct<count:string,name:string>>'),
 ('publication_day', 'string'),
 ('publication_month', 'string'),
 ('publication_year', 'string'),
 ('publisher', 'string'),
 ('ratings_count', 'string'),
 ('series', 'array<string>'),
 ('similar_books', 'array<string>'),
 ('text_reviews_count', 'string'),
 ('title', 'string'),
 ('title_without_series', 'string'),
 ('url', 'string'),
 ('work_id', 'string')]

* First of all, we will convert the datatypes of columns `average_rating`,`num_pages` & `ratings_count` from string to numeric type, in order to perfrom our calculations correctly.

In [561]:
from pyspark.sql.functions import *

df1 = df1.withColumn('average_rating',col('average_rating').cast('double'))
df1 = df1.withColumn('num_pages',col('num_pages').cast('double'))
df1 = df1.withColumn('ratings_count',col('ratings_count').cast('double'))

* Create a new dataframe `df2` which contains only the variables that we will use for our regression model.

In [562]:
df2 = df1.select(df1.average_rating, df1.language_code, df1.num_pages, df1.ratings_count, df1.publication_year)
df2.show()

+--------------+-------------+---------+-------------+----------------+
|average_rating|language_code|num_pages|ratings_count|publication_year|
+--------------+-------------+---------+-------------+----------------+
|          4.12|             |     null|          1.0|                |
|          3.94|          fre|     null|         16.0|            2016|
|          4.28|          eng|    146.0|         51.0|            2012|
|          4.05|          eng|     null|          6.0|                |
|          4.06|        en-US|    272.0|         51.0|            1997|
|          3.44|             |    206.0|         46.0|            2007|
|          4.15|          eng|    224.0|         39.0|            2016|
|          3.16|             |    160.0|         38.0|            2016|
|          3.51|             |    160.0|         44.0|            2016|
|           4.0|             |    144.0|         32.0|            2016|
|          4.41|          kor|    212.0|        133.0|          

* We will replace blank spaces in string variables (e.g., columns: `language_code` and `publication_year`) with "unknown" value.

In [563]:
replacements = {
  'language_code': 'unknown', 'publication_year': 'unknown',
  'numeric_column_wont_be_replaced': 1.0
}

for k, v in replacements.items():
    if isinstance(v, str):
        df2 = df2.na.replace("", v, [k])

df2.show()

+--------------+-------------+---------+-------------+----------------+
|average_rating|language_code|num_pages|ratings_count|publication_year|
+--------------+-------------+---------+-------------+----------------+
|          4.12|      unknown|     null|          1.0|         unknown|
|          3.94|          fre|     null|         16.0|            2016|
|          4.28|          eng|    146.0|         51.0|            2012|
|          4.05|          eng|     null|          6.0|         unknown|
|          4.06|        en-US|    272.0|         51.0|            1997|
|          3.44|      unknown|    206.0|         46.0|            2007|
|          4.15|          eng|    224.0|         39.0|            2016|
|          3.16|      unknown|    160.0|         38.0|            2016|
|          3.51|      unknown|    160.0|         44.0|            2016|
|           4.0|      unknown|    144.0|         32.0|            2016|
|          4.41|          kor|    212.0|        133.0|          

* We will drop the rows which contain Nulls.

In [566]:
df2 = df2.na.drop()
df2.count()

3617

## Train - Test split

* We will split our dataset in train and test dataset. The train dataset will be the 70% of the original dataset, while the rest 30% will be the test dataset, according to which we will evaluate the predictive performance of our model.

In [568]:
trainDF, testDF = df2.randomSplit([0.7, 0.3], seed=3) #We use random seed so in every run we have the same split on train test.

print(trainDF.cache().count()) #We use cache for efficiency (coumputational) reasons, as we will use the training set multiple times. 

print(testDF.count())

2490
1127


* In this section, we will perform one hot encoding, which converts categorical variables into a set of numeric variables that only take on values 0 and 1. The variables that we will transform are `language_code` & `publication_year`.

In [541]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

# We define which columns are categorical
categoricalCols = ["language_code" , "publication_year"]

stringIndexer = StringIndexer(inputCols=categoricalCols, outputCols=[x + "Index" for x in categoricalCols]).setHandleInvalid("keep") 
encoder = OneHotEncoder(inputCols=stringIndexer.getOutputCols(), outputCols=[x + "OHE" for x in categoricalCols]) 

In [542]:
stringIndexerModel = stringIndexer.fit(trainDF)
stringIndexerModel.transform(trainDF).show(7)

+--------------+-------------+---------+-------------+----------------+------------------+---------------------+
|average_rating|language_code|num_pages|ratings_count|publication_year|language_codeIndex|publication_yearIndex|
+--------------+-------------+---------+-------------+----------------+------------------+---------------------+
|           2.0|          ind|    128.0|          1.0|            2010|               4.0|                  6.0|
|           2.0|      unknown|    144.0|          1.0|            2017|               1.0|                  9.0|
|          2.35|      unknown|    104.0|         43.0|            2008|               1.0|                  8.0|
|          2.55|      unknown|    128.0|         33.0|            2004|               1.0|                 14.0|
|          2.59|      unknown|     96.0|         74.0|            2018|               1.0|                 36.0|
|          2.67|      unknown|     64.0|         12.0|            2003|               1.0|      

In [548]:
from pyspark.ml.feature import VectorAssembler

# This includes both the numeric columns and the one-hot encoded binary vector columns in our dataset.
numericCols = ["num_pages", "ratings_count"]
assemblerInputs = [c + "OHE" for c in categoricalCols] + numericCols
vecAssembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
# With VectorAssembler transformer we create a single vector column from assemblerInputs which contains the numeric and the categorical (e.g., with one-hot encoded binary variables) columns.

## Defining the model

* In this section, we will define our predictive model.

In [544]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(featuresCol = 'features', labelCol='average_rating', regParam=1.0)

## Pipeline

* First, we will define the pipeline based on the stages created in previous steps (e.g., `stringIndexer`, `encoder`, `vecAssembler` and `lr`.)

* Then we will define the pipeline model (e.g., `pipelineModel`).

* Last, we will aplly the pipeline model to the test dataset (e.g., `testDF`) to predict the `average_rating`).

In [545]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[stringIndexer, encoder, vecAssembler, lr])

pipelineModel = pipeline.fit(trainDF)

predDF = pipelineModel.transform(testDF)

* Now, we will display the predictions of our model.

In [546]:
predDF.select("features", "average_rating", "prediction").show(5)

+--------------------+--------------+------------------+
|            features|average_rating|        prediction|
+--------------------+--------------+------------------+
|(91,[1,42,89,90],...|          2.29|3.8669726947213894|
|(91,[1,54,89,90],...|          2.33|3.9029502464881274|
|(91,[1,39,89,90],...|          2.57| 3.864471457783849|
|(91,[1,42,89,90],...|          2.57|3.8823523274618634|
|(91,[0,42,89,90],...|          2.62| 3.908762360806099|
+--------------------+--------------+------------------+
only showing top 5 rows



* In order to evaluate the predictive performnace of our model, we will calculate the R Squared.

In [547]:
from pyspark.ml.evaluation import RegressionEvaluator

lr_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="average_rating",metricName="r2")

print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(predDF))

R Squared (R2) on test data = 0.0419129
